In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sleepdetector_claude import Sleepdetector
from tqdm import tqdm

# Load Data File
filepath = '../data/data.mat'
mat_file = sio.loadmat(filepath)
x = np.stack((mat_file['sig1'], mat_file['sig2'], mat_file['sig3'], mat_file['sig4']), axis=0)

# Convert numpy array to PyTorch tensor
x = torch.from_numpy(x).float()

# Load actual labels
labels_file = '../data/labels.mat'
y_true = sio.loadmat(labels_file)['labels'].flatten() - 1  # Subtract 1 to match 0-4 encoding
y = torch.from_numpy(y_true).long()

print(f"Shape of x: {x.shape}")
print(f"Shape of y: {y.shape}")

# Ensure x and y have matching dimensions
assert x.shape[1] == y.shape[0], f"Mismatch in number of examples: x has {x.shape[1]}, y has {y.shape[0]}"

# Create a custom dataset
class SleepDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, idx):
        return self.x[:, idx, :, :], self.y[idx]

#

Shape of x: torch.Size([4, 1027, 3000, 1])
Shape of y: torch.Size([1027])


In [2]:
 # Create dataset and dataloader
dataset = SleepDataset(x, y)
batch_size = 32
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the Sleepdetector
model = Sleepdetector()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [3]:
# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_x, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1} loss: {running_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y in tqdm(test_loader, desc="Evaluating"):
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
kappa = cohen_kappa_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)

print(f"Accuracy: {accuracy:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'trained_sleepdetector.pth')

Epoch 1/10:   0%|          | 0/26 [00:00<?, ?it/s]


RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [32, 1, 3000, 1]

In [5]:
labels.flatten()

array([5, 5, 5, ..., 3, 3, 2])

In [3]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import scipy.io
import numpy as np
import torch

# Load the .mat files
data_mat = scipy.io.loadmat('../data/data.mat')
labels_mat = scipy.io.loadmat('../data/labels.mat')

# Extract the signals
sig1 = data_mat['sig1']
sig2 = data_mat['sig2']
sig3 = data_mat['sig3']
sig4 = data_mat['sig4']

# Extract the labels
labels = labels_mat['labels']

# Convert to PyTorch tensors
sig1_tensor = torch.tensor(sig1, dtype=torch.float32)
sig2_tensor = torch.tensor(sig2, dtype=torch.float32)
sig3_tensor = torch.tensor(sig3, dtype=torch.float32)
sig4_tensor = torch.tensor(sig4, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Print the shapes to verify
print(f"sig1 shape: {sig1_tensor.shape}")
print(f"sig2 shape: {sig2_tensor.shape}")
print(f"sig3 shape: {sig3_tensor.shape}")
print(f"sig4 shape: {sig4_tensor.shape}")
print(f"labels shape: {labels_tensor.shape}")

# Create a TensorDataset and DataLoader
dataset = TensorDataset(sig1, sig2, sig3, sig4, labels)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

sig1 shape: torch.Size([1027, 3000, 1])
sig2 shape: torch.Size([1027, 3000, 1])
sig3 shape: torch.Size([1027, 3000, 1])
sig4 shape: torch.Size([1027, 3000, 1])
labels shape: torch.Size([1, 1027])


TypeError: 'int' object is not callable

In [ ]:
# train_sleep_detector.ipynb

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sleepdetector import SleepDetectorCNN

# Create the model
model = SleepDetectorCNN()

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example input data (replace with your actual data)
batch_size = 32
Fs = 100
n_classes = 5

sig1 = torch.randn(batch_size, 1, 30*Fs)
sig2 = torch.randn(batch_size, 1, 30*Fs)
sig3 = torch.randn(batch_size, 1, 30*Fs)
sig4 = torch.randn(batch_size, 1, 30*Fs)
labels = torch.randint(0, n_classes, (batch_size,))

# Create DataLoader
dataset = TensorDataset(sig1, sig2, sig3, sig4, labels)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

import torch

# Training loop
num_epochs = 25
best_loss = float('inf')
model_path = './sleep_detector_model.pth'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for signals, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(signals[0], signals[1], signals[2], signals[3])
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}")

    # Save the model weights if the loss has decreased
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), model_path)
        print(f"Model weights saved to {model_path}")

In [18]:
# train_sleep_detector.ipynb

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sleepdetector import SleepDetectorCNN
import scipy.io
import numpy as np

# Load the .mat files
data_mat = scipy.io.loadmat('../data/data.mat')
labels_mat = scipy.io.loadmat('../data/labels.mat')

# Extract the signals
sig1 = data_mat['sig1']
sig2 = data_mat['sig2']
sig3 = data_mat['sig3']
sig4 = data_mat['sig4']

# Extract the labels
labels = labels_mat['labels'].squeeze()

# Convert to PyTorch tensors
sig1_tensor = torch.tensor(sig1, dtype=torch.float32)
sig2_tensor = torch.tensor(sig2, dtype=torch.float32)
sig3_tensor = torch.tensor(sig3, dtype=torch.float32)
sig4_tensor = torch.tensor(sig4, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Print the shapes to verify
print(f"sig1 shape: {sig1_tensor.shape}")
print(f"sig2 shape: {sig2_tensor.shape}")
print(f"sig3 shape: {sig3_tensor.shape}")
print(f"sig4 shape: {sig4_tensor.shape}")
print(f"labels shape: {labels_tensor.shape}")

# Create a TensorDataset and DataLoader
dataset = TensorDataset(sig1_tensor, sig2_tensor, sig3_tensor, sig4_tensor, labels_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Create the model
model = SleepDetectorCNN()

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



sig1 shape: torch.Size([1027, 3000, 1])
sig2 shape: torch.Size([1027, 3000, 1])
sig3 shape: torch.Size([1027, 3000, 1])
sig4 shape: torch.Size([1027, 3000, 1])
labels shape: torch.Size([1027])


In [24]:
# training loop
num_epochs = 25
best_loss = float('inf')
model_path = './sleep_detector_model.pth'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for sig1, sig2, sig3, sig4, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(sig1, sig2, sig3, sig4)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}")

    # Save the model weights if the loss has decreased
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), model_path)
        print(f"Model weights saved to {model_path}")


RuntimeError: Given groups=1, weight of size [8, 1, 50], expected input[32, 3000, 2] to have 1 channels, but got 3000 channels instead

In [8]:
# Training loop
num_epochs = 10
best_loss = float('inf')
model_path = 'sleep_detector_model.pth'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for signals, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(signals[0], signals[1], signals[2], signals[3])
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}")

    # Save the model weights if the loss has decreased
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), model_path)
        print(f"Model weights saved to {model_path}")

# Loading the model weights for later use
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode

ValueError: too many values to unpack (expected 2)

In [20]:
# Training loop
num_epochs = 10
best_loss = float('inf')
model_path = 'sleep_detector_model.pth'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        # Unpack the signals and labels correctly
        sig1, sig2, sig3, sig4, labels = batch

        # Reshape the signals to the correct format (batch_size, 1, 3000)
        sig1 = sig1.permute(0, 2, 1)
        sig2 = sig2.permute(0, 2, 1)
        sig3 = sig3.permute(0, 2, 1)
        sig4 = sig4.permute(0, 2, 1)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(sig1, sig2, sig3, sig4)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}")

    # Save the model weights if the loss has decreased
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), model_path)
        print(f"Model weights saved to {model_path}")

# Loading the model weights for later use
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x640 and 5888x5)

In [17]:
# Training loop
num_epochs = 10
best_loss = float('inf')
model_path = 'sleep_detector_model.pth'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        # Unpack the signals and labels correctly
        sig1, sig2, sig3, sig4, labels = batch

        optimizer.zero_grad()

        # Forward pass
        outputs = model(sig1, sig2, sig3, sig4)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}")

    # Save the model weights if the loss has decreased
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), model_path)
        print(f"Model weights saved to {model_path}")

# Loading the model weights for later use
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode

RuntimeError: Given groups=1, weight of size [8, 1, 50], expected input[32, 3000, 2] to have 1 channels, but got 3000 channels instead

In [ ]:
# Visualize confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(5)
plt.xticks(tick_marks, ['Wake', 'N1', 'N2', 'N3', 'REM'], rotation=45)
plt.yticks(tick_marks, ['Wake', 'N1', 'N2', 'N3', 'REM'])
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
